In [202]:
import pandas as pd
from IPython.display import clear_output

In [236]:
quarter_1=pd.read_csv(r'C:\Users\79375\Desktop\jupyter projects\sber\sber 6.1\1_quarter_all.csv')
quarter_2=pd.read_csv(r'C:\Users\79375\Desktop\jupyter projects\sber\sber 6.1\2_quarter_all.csv')
quarter_3=pd.read_csv(r'C:\Users\79375\Desktop\jupyter projects\sber\sber 6.1\3_quarter_all.csv')
quarter_4=pd.read_csv(r'C:\Users\79375\Desktop\jupyter projects\sber\sber 6.1\4_quarter_all.csv')
train_ds=pd.read_csv(r'C:\\Users\\79375\\Desktop\\DS\\sber\\train.csv')
test_ds=pd.read_csv(r'C:\\Users\\79375\\Desktop\\DS\\sber\\sample_submission.csv')

test_ds['user_id'] = test_ds['id'].apply(lambda x: int(x.split(';')[0]))
test_ds['cart'] = test_ds['id'].apply(lambda x: int(x.split(';')[1]))
test_ds.drop('id', axis=1, inplace=True)
train_ds.drop('order_completed_at', axis=1, inplace=True)
train_ds['target'] = train_ds.duplicated(keep=0).apply(lambda x: 1 if x == 1 else 0)

In [237]:
def procent(id_tuple_test):
    progress=id_tuple_test.count()
    i=0
    procent=[]
    while i <=100 :
        procent.append(int(round((progress/100)*i,0)))
        i+=1
    return procent

In [238]:
user_id_train=pd.Series(train_ds['user_id'].drop_duplicates())
user_id_train=user_id_train.sort_values()
user_id_train

2422913        0
65467          1
0              2
16             3
18             4
           ...  
3049047    19995
3049764    19996
3053375    19997
3054305    19998
3074014    19999
Name: user_id, Length: 20000, dtype: int64

In [239]:
user_id_test=pd.Series(test_ds['user_id'].drop_duplicates())
user_id_test=user_id_test.sort_values()
user_id_test

0             0
36            1
56            3
94            4
144           5
          ...  
790343    19994
790372    19995
790381    19996
790398    19997
790436    19998
Name: user_id, Length: 13036, dtype: int64

In [240]:
customers_buy_map=[]
lap=1
process=procent(user_id_train)
for index in user_id_train:
    customer_cart_map=train_ds[train_ds['user_id']==index]
    customer_cart_map=customer_cart_map.drop_duplicates('cart')
    customers_buy_map.append(customer_cart_map)
    if index in process:
        clear_output()
        print(lap,' %')
        lap+=1

100  %


In [241]:
customers_buy_map_index_reset=[]
for i in customers_buy_map:
    reset= i.reset_index(drop=True)
    customers_buy_map_index_reset.append(reset)

In [242]:
vectors=[quarter_1,quarter_2,quarter_3,quarter_4]
global_vector_map=pd.concat(vectors)

In [243]:
global_vector_map_list=[]
for index in range(global_vector_map['1'].count()):
    vectors_map=global_vector_map.iloc[index]
    global_vector_map_list.append(list(pd.Series(vectors_map)))

In [244]:
# for index in range(test_ds['target'].count())
lap=1
predict=[]
process=procent(test_ds['target'])
for index in range(test_ds['target'].count()):
    user_id=test_ds.iloc[index]['user_id']
    user_cart=test_ds.iloc[index]['cart']
    target_counter=0
    cart_counert=0
    for vector in global_vector_map_list[user_id]:
        if user_cart in  list(pd.Series(customers_buy_map[vector]['cart'])):
            index_cart=customers_buy_map_index_reset[user_id][customers_buy_map_index_reset[user_id]['cart']==user_cart].index
            target=customers_buy_map_index_reset[user_id].iloc[index_cart]['target']
            target=int(target)
            if target==1:
                target_counter+=1
            cart_counert+=1
    if cart_counert!=0:
        predict.append(1 if target_counter/cart_counert>0.4 else 0)
    else:
        predict.append(0)
    if index in process:
        clear_output()
        print(lap,' %')
        lap+=1
    
# predict
        

100  %


In [249]:
test_ds1=test_ds.drop('target', axis=1)
test_ds1['target']=predict
test_ds1


user_id  cart  target
0             0   133       0
1             0     5       0
2             0    10       0
3             0   396       0
4             0    14       1
...         ...   ...     ...
790444    19998    26       0
790445    19998    31       0
790446    19998    29       0
790447    19998   798       0
790448    19998   415       1

[790449 rows x 3 columns]

In [250]:
def scv_submiss(predictions, files_name='Standart_name'):
    id_cart=test_ds.user_id.apply(str) + ';' + test_ds.cart.apply(str)
    submission = pd.DataFrame({'id': id_cart, 'target': predictions})
    submission.to_csv(files_name+'.csv', index=False)

In [251]:
scv_submiss(predict,files_name='sber_6.1_submis')